In [1]:
import uuid
import json
import redis
import paho.mqtt.client as mqtt 

#connect to a broker to send/receive messages to/from the broker
client = mqtt.Client()


series1 = "battery"
series2 = "power"

print("Insert Redis host")
REDIS_HOST = input()
print("Insert Redis port")
REDIS_PORT = int(input()) 
REDIS_USER = "default"
print("Insert Redis Password")
REDIS_PASSWORD = input()

redis_client = redis.Redis(host = REDIS_HOST, port = REDIS_PORT, password = REDIS_PASSWORD, username= REDIS_USER)
print("Is connected:", redis_client.ping())


Is connected: True


In [2]:
def check_series_creation(redis_client, mac_address):
    '''checks for the existence of the series and creates them'''
    try:
        redis_client.ts().create(mac_address+":battery", label = {"second" : "battery"} )
    except redis.ResponseError:
        pass
    try:
        redis_client.ts().create(mac_address+":power")
    except redis.ResponseError:
        pass


def on_connect(client, userdata, flags, rc):
    #subscribe to a topic
    client.subscribe('s291871')


def on_message(client, userdata, msg):
    message = msg.payload.decode() 
    message_dict = json.loads(message)

    #whenever a new mac_address is received, we need to create the redis timeseries
    check_series_creation(redis_client, message_dict["mac_address"])
    
    #store the data
    redis_client.ts().add(message_dict["mac_address"]+":battery", message_dict["timestamp"], message_dict["battery_level"])
    redis_client.ts().add(message_dict["mac_address"]+":power", message_dict["timestamp"], message_dict["power_plugged"])



#bind the on_connect and on_message method to the client
client.on_connect = on_connect
client.on_message = on_message

#connect to the broker 
client.connect('mqtt.eclipseprojects.io', 1883)

0

In [3]:
#keep listening over the topic we subscribed to using an infinite loop 
client.loop_forever()

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1e2d0164-f620-4188-849f-f2bd90b2ba46' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>